In [1]:
import numpy as np
import torch
from torch import nn
from torchvision import transforms
from torch.utils.data import DataLoader
import cv2
import random
import os

In [2]:
def data_preprocessing(path,categories):
    train_data=[]
    trans=transforms.Compose([transforms.ToTensor()])
    for category in categories:
        index=categories.index(category)
        train_path=os.path.join(path,category)
        for img in os.listdir(train_path):
            img_to_array=cv2.imread(os.path.join(train_path,img),cv2.IMREAD_GRAYSCALE)
            resized=cv2.resize(img_to_array,(150,150))
            train_data.append([trans(resized),index])
    random.shuffle(train_data)
    return train_data

In [3]:
def data_loader(data,batch_size):
    BATCH=DataLoader(data,batch_size=batch_size)
    return BATCH

In [4]:
class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Sequential(nn.Conv2d(in_channels=1,out_channels=32,kernel_size=2,stride=1,padding=2),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2))
        self.conv2=nn.Sequential(nn.Conv2d(in_channels=32,out_channels=64,kernel_size=2,stride=1,padding=2),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2))
        self.conv3=nn.Sequential(nn.Conv2d(in_channels=64,out_channels=128,kernel_size=2,stride=1,padding=2),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2))
        self.flatten=nn.Flatten()
        self.linear1=nn.Linear(56448,128)
        self.relu1=nn.ReLU()
        self.linear2=nn.Linear(128,6)
        self.softmax=nn.Softmax(dim=1)
    def forward(self,input_data):
        x=self.conv1(input_data)
        x=self.conv2(x)
        x=self.conv3(x)
        x=self.flatten(x)
        x=self.linear1(x)
        x=self.relu1(x)
        x=self.linear2(x)
        prediction=self.softmax(x)
        return prediction

In [5]:
def train_one_epoch(model,data,loss_fn,optizer,device):
    for input_,target in data:
        input_,target=input_.to(device),target.to(device)
        prediction=model(input_)
        loss=loss_fn(prediction,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(loss.item())
def train(model,data,loss_fn,optimizer,device,epoch):
    for i in range(epoch):
        train_one_epoch(model,data,loss_fn,optimizer,device)

In [6]:
if torch.cuda.is_available():
    device='cuda'
else:
    device='cpu'

In [7]:
feed_forward=FeedForward().to(device)

In [8]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(feed_forward.parameters(),lr=0.001)

In [9]:
path="/Users/vatsa/OneDrive/Documents/IntelClassification/seg_train/seg_train/"
categories=['mountain','buildings','street','sea','glacier','forest']

In [10]:
Train_DATA=data_preprocessing(path,categories)

In [11]:
BATCH=data_loader(Train_DATA,100)

In [ ]:
train(feed_forward,BATCH,loss_fn,optimizer,device,5)